In [1]:
%cd ..

# Bayesian optimization: finding the minimum emission

In this notebook we will performing bayesian optimization on the emulator trained on our sumo simulation. We are interested in identifying the parameters which enable us to obtain the minimum travel time. Our emulator is a gaussian process with an **rbf** kernel.


## Imports

In [2]:
import pickle
import numpy as np
import emukit as ek
import GPy
import pandas as pd

from emukit.model_wrappers import GPyModelWrapper
from emukit.core.initial_designs import RandomDesign
from emukit.core import ParameterSpace, ContinuousParameter, DiscreteParameter
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement
from emukit.bayesian_optimization.loops import BayesianOptimizationLoop

from sumo_grid_simulation.grid_simulation import Simulator

warning in stationary: failed to import cython module: falling back to numpy
warning in coregionalize: failed to import cython module: falling back to numpy
warning in choleskies: failed to import cython module: falling back to numpy


## Simulator

We start by initialising our simulation.

In [3]:
simulator = Simulator(end_time=300)

We then define the two user functions in which we are interested:

In [4]:
def user_function_emissions(X):
    """  X = inputs - emukit doesnt pass named args, just an NxM ndarray, N is the number of points to evaluate, M is the number of parameters per each point """
    result = []
    i = 0
    
    print(X)
    print(f'\nUser function emissions called with {X.shape[0]} inputs to simulate')

    
    for gridSize, edgeMaxSpeed, edgeLength, numLanes, accel in X:
        print(f'\nEvaluating input: {i+1} of {X.shape[0]}\n')
        
        beta = 0.05
        max_number_of_vehicles = ((gridSize - 1) * gridSize * 2 + 4 * gridSize) * edgeLength / 5
        period = 300/(max_number_of_vehicles * beta)
        
        s = simulator.simulate(
            gridSize      = int(gridSize),
            edgeMaxSpeed  = edgeMaxSpeed,
            edgeLength    = edgeLength,
            numberOfLanes = int(numLanes),
            accel         = accel,
            trips_generator_period = period
        )
        

        CO2 = s['CO2']/s['num_emissions_samples']
        result.append(CO2/s['duration'])
        print(f'\nOutput {result[-1]}\n')
        i += 1
        
    # expand dims is essential or the acquition function breaks
    return np.expand_dims(np.array(result), 1)  

We then load the parameters' space from our configuration file:

In [5]:
gridSize = DiscreteParameter('gridSize', domain=[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])
edgeMaxSpeed = ContinuousParameter('edgeMaxSpeed', min_value=8, max_value=19)
edgeLength = ContinuousParameter('edgeLength', min_value=30, max_value=70)
numberOfLanes = DiscreteParameter('numberOfLanes', domain=[1,2,3])
accel = ContinuousParameter('accel', 1.5, 5.)

parameter_space = ParameterSpace([gridSize, edgeMaxSpeed, edgeLength, numberOfLanes, accel])
#parameter_space = config.get_parameter_space()

## Emulator

### Rnadomly initialised emulator

The first emulator we are going to analyse is the one that is just initilised with random points

We first sample at random 200 datapoints from the parameter space:

In [6]:
#init_X, init_Y = pickle.load(open('bayesian_optimization/init_points/270_init_points_emissions.pkl', 'rb'))


design = RandomDesign(parameter_space)
num_init_points = 520
init_X = design.get_samples(num_init_points)
init_Y = user_function_emissions(init_X)
init_points = init_X, init_Y
with open(f'bayesian_optimization/init_points/{num_init_points}_init_points_emissions.pkl', "wb") as f:
     pickle.dump(init_points, f)


[[18.         18.37580594 66.37890813  3.          3.31614634]
 [14.          8.75403839 52.68674583  3.          4.94583528]
 [16.         15.05951332 49.95053746  3.          3.93313036]
 ...
 [14.         13.99416273 36.41058713  1.          3.96060821]
 [16.         14.05766707 51.12358668  3.          3.20878383]
 [ 8.         16.64026243 52.14930414  2.          2.1162512 ]]

User function emissions called with 520 inputs to simulate

Evaluating input: 1 of 520

 Retrying in 1 seconds

Output 52.63334827480334


Evaluating input: 2 of 520

 Retrying in 1 seconds

Output 25.90067723402686


Evaluating input: 3 of 520

 Retrying in 1 seconds

Output 62.82487387411132


Evaluating input: 4 of 520

 Retrying in 1 seconds

Output 44.32452194510021


Evaluating input: 5 of 520

 Retrying in 1 seconds

Output 154.69971407210704


Evaluating input: 6 of 520

 Retrying in 1 seconds

Output 25.21387405565659


Evaluating input: 7 of 520

 Retrying in 1 seconds

Output 99.27143083428332


E


Output 46.13283465285698


Evaluating input: 98 of 520

 Retrying in 1 seconds

Output 311.7583561812638


Evaluating input: 99 of 520

 Retrying in 1 seconds

Output 62.27714112184195


Evaluating input: 100 of 520

 Retrying in 1 seconds

Output 45.654377930625


Evaluating input: 101 of 520

 Retrying in 1 seconds

Output 30.97078559802079


Evaluating input: 102 of 520

 Retrying in 1 seconds

Output 40.74270950269857


Evaluating input: 103 of 520

 Retrying in 1 seconds

Output 36.96263602325662


Evaluating input: 104 of 520

 Retrying in 1 seconds

Output 76.18870875689545


Evaluating input: 105 of 520

 Retrying in 1 seconds

Output 87.15515097842838


Evaluating input: 106 of 520

 Retrying in 1 seconds

Output 70.46064768717113


Evaluating input: 107 of 520

 Retrying in 1 seconds

Output 81.17969208110188


Evaluating input: 108 of 520

 Retrying in 1 seconds

Output 90.63698921788435


Evaluating input: 109 of 520

 Retrying in 1 seconds

Output 32.36054717133241


Eval


Output 51.22754594435889


Evaluating input: 199 of 520

 Retrying in 1 seconds

Output 75.38874934208366


Evaluating input: 200 of 520

 Retrying in 1 seconds

Output 70.27768590306977


Evaluating input: 201 of 520

 Retrying in 1 seconds

Output 84.08521475259214


Evaluating input: 202 of 520

 Retrying in 1 seconds

Output 51.36181317622481


Evaluating input: 203 of 520

 Retrying in 1 seconds

Output 25.596923859411238


Evaluating input: 204 of 520

 Retrying in 1 seconds

Output 199.41100079928978


Evaluating input: 205 of 520

 Retrying in 1 seconds

Output 229.78538649128564


Evaluating input: 206 of 520

 Retrying in 1 seconds

Output 39.91059389432528


Evaluating input: 207 of 520

 Retrying in 1 seconds

Output 44.413745024875716


Evaluating input: 208 of 520

 Retrying in 1 seconds

Output 14.978295627636236


Evaluating input: 209 of 520

 Retrying in 1 seconds

Output 23.070569619137515


Evaluating input: 210 of 520

 Retrying in 1 seconds

Output 85.50178921799


Output 44.96521114559585


Evaluating input: 300 of 520

 Retrying in 1 seconds

Output 51.17474070511112


Evaluating input: 301 of 520

 Retrying in 1 seconds

Output 59.036853879455606


Evaluating input: 302 of 520

 Retrying in 1 seconds

Output 65.97963583343567


Evaluating input: 303 of 520

 Retrying in 1 seconds

Output 18.288531079810635


Evaluating input: 304 of 520

 Retrying in 1 seconds

Output 118.52912401535684


Evaluating input: 305 of 520

 Retrying in 1 seconds

Output 38.615736815571445


Evaluating input: 306 of 520

 Retrying in 1 seconds

Output 37.13429980363706


Evaluating input: 307 of 520

 Retrying in 1 seconds

Output 81.03113897440979


Evaluating input: 308 of 520

 Retrying in 1 seconds

Output 18.767855616771104


Evaluating input: 309 of 520

 Retrying in 1 seconds

Output 126.82503570355917


Evaluating input: 310 of 520

 Retrying in 1 seconds

Output 37.503563451258174


Evaluating input: 311 of 520

 Retrying in 1 seconds

Output 37.6257955034


Output 84.6975484079764


Evaluating input: 401 of 520

 Retrying in 1 seconds

Output 43.37984720256679


Evaluating input: 402 of 520

 Retrying in 1 seconds

Output 41.468150696767786


Evaluating input: 403 of 520

 Retrying in 1 seconds

Output 16.969611423279495


Evaluating input: 404 of 520

 Retrying in 1 seconds

Output 57.2231090752894


Evaluating input: 405 of 520

 Retrying in 1 seconds

Output 193.41246596815432


Evaluating input: 406 of 520

 Retrying in 1 seconds

Output 123.07077821449327


Evaluating input: 407 of 520

 Retrying in 1 seconds

Output 44.91392182364876


Evaluating input: 408 of 520

 Retrying in 1 seconds

Output 32.25026286445576


Evaluating input: 409 of 520

 Retrying in 1 seconds

Output 48.10335785887369


Evaluating input: 410 of 520

 Retrying in 1 seconds

Output 44.37463735011937


Evaluating input: 411 of 520

 Retrying in 1 seconds

Output 61.971859627023925


Evaluating input: 412 of 520

 Retrying in 1 seconds

Output 171.4843802540822


Output 64.59485846034698


Evaluating input: 502 of 520

 Retrying in 1 seconds

Output 51.57051787535656


Evaluating input: 503 of 520

 Retrying in 1 seconds

Output 40.97520145967839


Evaluating input: 504 of 520

 Retrying in 1 seconds

Output 101.35796048526856


Evaluating input: 505 of 520

 Retrying in 1 seconds

Output 36.41154474267219


Evaluating input: 506 of 520

 Retrying in 1 seconds

Output 29.852508478378976


Evaluating input: 507 of 520

 Retrying in 1 seconds

Output 37.64978925227397


Evaluating input: 508 of 520

 Retrying in 1 seconds

Output 129.5006596539225


Evaluating input: 509 of 520

 Retrying in 1 seconds

Output 111.01488833410203


Evaluating input: 510 of 520

 Retrying in 1 seconds

Output 119.0693171436974


Evaluating input: 511 of 520

 Retrying in 1 seconds

Output 75.24555143776456


Evaluating input: 512 of 520

 Retrying in 1 seconds

Output 112.13155231630503


Evaluating input: 513 of 520

 Retrying in 1 seconds

Output 47.9544861291920

To then fit a GP to these points

In [7]:
emulator = GPy.models.GPRegression(init_X, init_Y, noise_var=1e-10)
emukit_model_random_init = GPyModelWrapper(emulator, n_restarts=5)
emukit_model_random_init.optimize()

Optimization restart 1/5, f = 1975.6097017017055
Optimization restart 2/5, f = 1717.2112589075805
Optimization restart 3/5, f = 1717.2112589161657
Optimization restart 4/5, f = 1717.2112588812688
Optimization restart 5/5, f = 1717.211258909081


### Experimentally designed emulator with model variance

We also load the emulator obtained during experimental design:

In [8]:
emukit_model_variance = pickle.load(open('experimental_design/models/5param_model_variance_20_init_points_500_loops_co2_per_duration.pkl', 'rb'))

### Experimentally designed emulator with integrated variance reduction

We also load the emulator obtained during experimental design:

In [9]:
emukit_model_integrated_variance = pickle.load(open('experimental_design/models/5param_integrated_variance_reduction_20_init_points_500_loops_co2_per_duration.pkl', 'rb'))

## Bayesian optimisation

We now run bayesian optimisation on the emulators

In [10]:
n_iter_bo = 50

### Acquisition functions

In [11]:
acquisition_random_init = ExpectedImprovement(emukit_model_random_init)

In [12]:
acquisition_model_variance = ExpectedImprovement(emukit_model_variance)

In [13]:
acquisition_integrated_variance = ExpectedImprovement(emukit_model_integrated_variance)

### Optimisation loop

In [14]:
bo_random_init = BayesianOptimizationLoop(parameter_space, emukit_model_random_init, acquisition=acquisition_random_init)
bo_random_init.run_loop(user_function_emissions, n_iter_bo)

 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 1717.2112588812688
Optimization restart 2/5, f = 1717.2112589623266
Optimization restart 3/5, f = 1717.211259060177
Optimization restart 4/5, f = 1717.2112589234437
Optimization restart 5/5, f = 1717.2112597995704
[[18.          8.         48.46094505  1.          1.5       ]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 16.624623101607494



 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 1720.5724734731252
Optimization restart 2/5, f = 1720.572181089012
Optimization restart 3/5, f = 1720.5721811902158
Optimization restart 4/5, f = 1720.572181297566
Optimization restart 5/5, f = 1720.5721871762094
[[18.  8. 70.  3.  5.]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 12.556290419595172



 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


...

Optimization restart 2/5, f = 3703.4998978688027
Optimization restart 3/5, f = 1832.464766799716
Optimization restart 4/5, f = 1832.4647659963716
Optimization restart 5/5, f = 1832.4647660168998
[[19.   8.  70.   1.   1.5]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 10.012610845527409



 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 1834.5957177944947
Optimization restart 2/5, f = 1834.5955120474023
Optimization restart 3/5, f = 1834.5955120610233
Optimization restart 4/5, f = 1834.5955120435963
Optimization restart 5/5, f = 1834.5955120375465
[[19.   8.  70.   1.   1.5]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 10.012610845527409



 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 1836.7232367804404
Optimization restart 2/5, f = 1836.7230173519142
Optimization restart 3/5, f = 1836.7230173447365
Optimization restart 4/5, f = 1836.7230173377188
Optimization restart 5/5, f = 1836.7230173335836


In [15]:
bo_model_variance = BayesianOptimizationLoop(parameter_space, emukit_model_variance, acquisition=acquisition_model_variance)
bo_model_variance.run_loop(user_function_emissions, n_iter_bo)

Optimization restart 1/5, f = 1932.6432341044756
Optimization restart 2/5, f = 1954.2447248734327


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


Optimization restart 3/5, f = 1932.6432341051682
Optimization restart 4/5, f = 1932.6432341159634
Optimization restart 5/5, f = 1932.6432341109924
[[18.   8.  70.   1.   1.5]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 10.982837146006856



 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 1935.4230770308013
Optimization restart 2/5, f = 1956.5925339077894


...

Optimization restart 1/5, f = 2030.3263187859745
Optimization restart 2/5, f = 2030.3261552994045
Optimization restart 3/5, f = 2030.326195377176
Optimization restart 4/5, f = 2058.5989939599285
Optimization restart 5/5, f = 2059.356385023137
[[18.          8.98266645 67.85932763  3.          1.5       ]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 14.794730015287328



 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 2031.8884094551975
Optimization restart 2/5, f = 2031.8882142572716
Optimization restart 3/5, f = 2031.8882942106382
Optimization restart 4/5, f = 2031.8882038432566
Optimization restart 5/5, f = 2061.1335715704445
[[18.          8.         47.45616493  3.          1.5       ]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 18.12748305205415



 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 2034.1532851500506
Optimization restart 2/5, f = 2034.1524478655367
Optimization restart 3/5, f = 2063.8935233217026
Optimization restart 4/5, f = 2034.152811248483
Optimization restart 5/5, f = 2034.152867192927


In [16]:
bo_integrated_variance = BayesianOptimizationLoop(parameter_space, emukit_model_integrated_variance, acquisition=acquisition_integrated_variance)
bo_integrated_variance.run_loop(user_function_emissions, n_iter_bo)

Optimization restart 1/5, f = 1743.4672042288594
Optimization restart 2/5, f = 1743.4672042306831
Optimization restart 3/5, f = 1743.4672042357477
Optimization restart 4/5, f = 1743.467204730262
Optimization restart 5/5, f = 1743.4672042294028
[[18.         8.        65.0318202  1.         1.5      ]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 11.993070069034932



 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 1746.763630219466
Optimization restart 2/5, f = 1746.7634606325794
Optimization restart 3/5, f = 1746.7634606342913
Optimization restart 4/5, f = 1746.7634606331824


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 5/5, f = 2565.542220638123
[[20.   8.  70.   3.   1.5]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 10.25551968350352



...

Optimization restart 1/5, f = 1856.0178045155685
Optimization restart 2/5, f = 1856.0177562644008
Optimization restart 3/5, f = 1856.0177627044177
Optimization restart 4/5, f = 1856.017756444017
Optimization restart 5/5, f = 1856.0177562898489
[[20.          8.         69.15510218  1.          1.5       ]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 9.908390231137343



 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 1858.1267217676973
Optimization restart 2/5, f = 1858.126679708176
Optimization restart 3/5, f = 1858.1266792559509
Optimization restart 4/5, f = 1858.1266791028524
Optimization restart 5/5, f = 1858.1266790956165
[[20.   8.  70.   1.   1.5]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 9.79286440956895



 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 1860.2286561250626
Optimization restart 2/5, f = 1860.2286091109138
Optimization restart 3/5, f = 1860.2286091108535
Optimization restart 4/5, f = 1860.2286091250623
Optimization restart 5/5, f = 1860.2286091071464


## Results

In [17]:
parameters_name = ['Size of the grid', 'Speed limit in the net (m/s)', 'Length of the roads (m)', 'Number of lanes', 'Accelleration (m/s^2)']

### Results on the random initialised emulator

In [18]:
results_random_init = bo_random_init.get_results()

In [19]:
results_random_init_df = pd.DataFrame(results_random_init.minimum_location.reshape(1,-1))
results_random_init_df.columns = parameters_name
results_random_init_df

,Size of the grid,Speed limit in the net (m/s),Length of the roads (m),Number of lanes,Accelleration (m/s^2)
0,19.0,8.0,70.0,1.0,1.5


In [20]:
results_random_init.minimum_value

10.012610845527409

### Results on the experimentally designed emulator with model variance

In [21]:
results_model_variance = bo_model_variance.get_results()

In [22]:
results_random_init_df = pd.DataFrame(results_model_variance.minimum_location.reshape(1,-1))
results_random_init_df.columns = parameters_name
results_random_init_df

,Size of the grid,Speed limit in the net (m/s),Length of the roads (m),Number of lanes,Accelleration (m/s^2)
0,20.0,8.0,70.0,1.0,1.5


In [23]:
results_model_variance.minimum_value

9.79286440956895

### Results on the experimentally designed emulator with integrated variance reduction

In [24]:
results_integrated_variance = bo_integrated_variance.get_results()

In [25]:
results_random_init_df = pd.DataFrame(results_integrated_variance.minimum_location.reshape(1,-1))
results_random_init_df.columns = parameters_name
results_random_init_df

,Size of the grid,Speed limit in the net (m/s),Length of the roads (m),Number of lanes,Accelleration (m/s^2)
0,20.0,8.0,70.0,1.0,1.5


In [26]:
results_integrated_variance.minimum_value

9.79286440956895

## Save stuff

In [27]:
with open(f'bayesian_optimization/bayesian_opt_results/results_random_init_emissions.pkl', "wb") as f:
     pickle.dump(results_random_init, f)

with open(f'bayesian_optimization/bayesian_opt_results/results_model_variance_emissions.pkl', "wb") as f:
     pickle.dump(results_model_variance, f)
        
with open(f'bayesian_optimization/bayesian_opt_results/results_integrated_variance_emissions.pkl', "wb") as f:
     pickle.dump(results_integrated_variance, f)

In [28]:
with open(f'bayesian_optimization/models/520_random_init_50_bay_opt_emissions.pkl', "wb") as f:
     pickle.dump(emukit_model_random_init, f)

with open(f'bayesian_optimization/models/520_model_variance_50_bay_emissions.pkl', "wb") as f:
     pickle.dump(emukit_model_variance, f)
        
with open(f'bayesian_optimization/models/520_integrated_variance_50_bay_emissions.pkl', "wb") as f:
     pickle.dump(emukit_model_integrated_variance, f)